In [35]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree
from nltk import ngrams
import json

In [36]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import extract_info
import parameters as params
import download_and_process_pdf as dpp

In [37]:
import shutil

test_path = fpath.poten_litera_testing_set_300_read_index_corrected
# destination = fpath.relevant_pdf_folder
destination = "/media/hou/DIDIHOU/relevant_pdfs"

df_test = pd.read_csv(test_path, header=0, sep=',')
df_test.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS", "RELEVANT"]

for ind in df_test.index:
    index = df_test.at[ind, "INDEX"]
    if df_test.at[ind, "RELEVANT"] == "YES":
        print(ind, index)
        json_path = os.path.join(fpath.pdf_folder, str(index) + ".json")
        pdf_path = os.path.join(fpath.pdf_folder, str(index) + ".pdf")
        
        if os.path.exists(json_path):
            shutil.copy(json_path, destination)
        if os.path.exists(pdf_path):
            shutil.copy(pdf_path, destination)
    else:
        pass

23 619
43 220
44 3209
62 784
89 2756
105 394
107 3520
119 820
143 3625
151 851
152 5905
153 4077
155 223
160 793
164 85
167 941
170 4
172 377
176 134
208 691
226 1964
229 1147
247 479
256 2440
270 4005
